In [116]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import csv

path = r'C:\Users\tiany\Documents\EE6895 big data\bill\110'
allFiles = glob.glob(path + "/*.csv")
#vote same or not between party and individuals
Party =[]
Party_vote = []
position = []
selfvote = []
#contribution differences
difference = []


#try legislator 'Neil Abercrombie''Democrate'/ 'Stephanie Herseth Sandlin

for i in range(len(allFiles)):
    df = pd.read_csv(allFiles[i], encoding='latin-1',low_memory=False)
    #print(allFiles[i])
    df['Contribution Amount'] = df['Contribution Amount'].str.replace('$','')
    df['Contribution Amount'] = df['Contribution Amount'].astype(float)
    name = df.columns.values.tolist()
    v = name[3]
    n = name[-4]
    df = df.rename(columns={n: 'Interest Group Position',v:'Vote'})
    clean = df.drop(['Represents','Contribution Type','Presidential','Contribution Date','Contributor','Contributor Occupation','Contributor Interest Group Code','Contributor City','Contributor State','Contributor Zip','Contributor Employer','Contributor Interest Group'],axis=1)
    g = clean.groupby(['Party','Interest Group Position']).sum()
    newclean =  df.drop(['Represents','Contribution Type','Presidential','Contribution Amount','Contribution Date','Contributor','Contributor Occupation','Contributor Interest Group Code','Contributor City','Contributor State','Contributor Zip','Contributor Employer','Contributor Interest Group'],axis=1)
    dd = newclean.drop_duplicates(subset=['Legislator', 'Party', 'Vote'])
    a = (dd['Party'][dd['Vote']=='No']=='Democrat').sum()
    b = (dd['Party'][dd['Vote']=='Not Voting']=='Democrat').sum()
    c = (dd['Party'][dd['Vote']=='Yes']=='Democrat').sum()
    d = (dd['Party'][dd['Vote']=='No']=='Independent').sum()
    e = (dd['Party'][dd['Vote']=='Not Voting']=='Independent').sum()
    f = (dd['Party'][dd['Vote']=='Yes']=='Independent').sum()
    j = (dd['Party'][dd['Vote']=='No']=='Republican').sum()
    k = (dd['Party'][dd['Vote']=='Not Voting']=='Republican').sum()
    l = (dd['Party'][dd['Vote']=='Yes']=='Republican').sum()

    #dataframe
    pd.set_option('display.precision',15)
    pd.DataFrame({'x':[.000000000000001]})

    arrays = [['Democrat', 'Democrat','Democrat', 'Republican','Republican', 'Republican'],
                          ['Oppose','', 'Support', 'Oppose','', 'Support']]
    tuples = list(zip(*arrays))
    index = pd.MultiIndex.from_tuples(tuples, names=['Party', 'Interest Group Position'])
    column = ['Contribution Amount', 'Vote', 'Count']
    s = pd.DataFrame(index=index, columns=column)
    votes = ['No','Not Voting','Yes','No','Not Voting','Yes']
    count = [a,b,c,j,k,l]
    s['Vote'] = votes
    s['Count'] = count
    newframe = s.fillna(g)
    effect = newframe.fillna(0)
    gg = clean.groupby(['Legislator','Party','Vote','Interest Group Position']).sum()
    leglist = newclean.drop_duplicates(subset=['Legislator'])
    leglist = leglist['Legislator'].tolist()
    num = len(leglist)
    reset = gg.reset_index()
    try:
        name = reset[reset['Legislator']=='Tom Allen']
        if name['Party'].any() == 'Democrat':
            Party.append(1)
            con_oppo = name['Contribution Amount'][name['Interest Group Position']=='Oppose'].tolist()
            if con_oppo ==[]:
                con_oppo = 0.0
            else:
                con_oppo = con_oppo[0]
            con_sup = name['Contribution Amount'][name['Interest Group Position']=='Support'].tolist()
            if con_sup ==[]:
                con_sup = 0.0
            else:
                con_sup = con_sup[0]
            
            #party veto No
            if a > c:
                Party_vote.append(0)
            #party veto Yes
            if a < c:
                Party_vote.append(1)
            
            #position
            if con_sup > con_oppo:
                dif = con_sup - con_oppo
                difference.append(dif)
                position.append(1)
            if con_sup < con_oppo:
                dif = con_sup - con_oppo
                difference.append(dif)
                position.append(0)
            if con_sup == con_oppo:
                difference.append(0)
                position.append(2)
            
            if name.iloc[0]['Vote'] =='No':
                selfvote.append(0)
            if name.iloc[0]['Vote'] =='Yes':
                selfvote.append(1)
            if name.iloc[0]['Vote'] =='Not Voting':
                selfvote.append(2)
        if name['Party'].any() == 'Republican':
            Party.append(1)
            con_oppo = name['Contribution Amount'][name['Interest Group Position']=='Oppose'].tolist()
            if con_oppo ==[]:
                con_oppo = 0.0
            else:
                con_oppo = con_oppo[0]
            con_sup = name['Contribution Amount'][name['Interest Group Position']=='Support'].tolist()
            if con_sup ==[]:
                con_sup = 0.0
            else:
                con_sup = con_sup[0]
            
            #party veto No
            if a > c:
                Party_vote.append(0)
            #party veto Yes
            if a < c:
                Party_vote.append(1)
            
            #position
            if con_sup > con_oppo:
                dif = con_sup - con_oppo
                difference.append(dif)
                position.append(1)
            if con_sup < con_oppo:
                dif = con_sup - con_oppo
                difference.append(dif)
                position.append(0)
            if con_sup == con_oppo:
                difference.append(0)
                position.append(2)
            
            if name.iloc[0]['Vote'] =='No':
                selfvote.append(0)
            if name.iloc[0]['Vote'] =='Yes':
                selfvote.append(1)
            if name.iloc[0]['Vote'] =='Not Voting':
                selfvote.append(2)
    except:
        pass

columns = ['Party','Party Vote', 'Group Position','Contribution difference','self Vote']
mdf = pd.DataFrame(columns=columns)    
mdf['Party'] =  Party
mdf['Party Vote'] = Party_vote
mdf['Group Position'] = position
mdf['Contribution difference'] = difference
mdf['self Vote'] = selfvote

C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [115]:
len(difference)

217

mdf

In [117]:
mdf['is_train'] = np.random.uniform(0, 1, len(mdf)) <= .75

In [118]:
mdf

,Party,Party Vote,Group Position,Contribution difference,self Vote,is_train
0,1,1,1,123288.0,1,True
1,1,1,1,4800.0,1,True
2,1,1,1,17000.0,1,False
3,1,1,1,545147.0,1,True
4,1,1,1,42675.0,1,True
5,1,1,1,500.0,1,True
6,1,1,0,-15600.0,2,False
7,1,1,0,-27040.0,1,True
8,1,1,1,24290.0,1,True
9,1,1,1,176683.0,1,True


In [119]:
train, test = mdf[mdf['is_train']==True], mdf[mdf['is_train']==False]
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 170
Number of observations in the test data: 48


In [120]:
features = mdf.columns[:4]
y = np.array(train['self Vote'])
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(train[features], y)
predict = clf.predict(test[features])
og = np.array(test['self Vote'])
bt=0
for i in range(len(og)):
    if predict[i] == og[i]:
        bt+=1
p = bt/len(og)
print('prob of success :',p)

prob of success : 0.9791666666666666


In [121]:
data_prob = clf.predict_proba(test[features])[:]
pd.set_option('display.precision',1)
probability =  pd.DataFrame(data = data_prob,columns=['No', 'Yes', 'Not Voting'])
pathf = r'C:\Users\tiany\Documents\EE6895 big data\bill\ '
probability.to_csv(pathf+'Tom Allen predict.csv') 

In [122]:
probability

,No,Yes,Not Voting
0,0.0,1.0,0.0
1,0.2,0.8,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
5,0.0,1.0,0.0
6,0.0,1.0,0.0
7,0.0,1.0,0.0
8,0.0,1.0,0.0
9,0.0,1.0,0.0
